In [14]:
from email.mime.multipart import MIMEMultipart
import os.path
import pandas as pd
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import datetime as dt
from email.mime.text import MIMEText
import base64

In [35]:
# Connect to Drive API
spreadsheet_id = '1sSuQo8XTjcbeYyCXtGn-eYl5TJ-ZMpC0BaakwzwUoRY'
url = 'https://docs.google.com/spreadsheets/d/1-2FvSx-rVma1RCZ0Hu4p7J8DJjN6TpZwz6CgNSwa7CY'
emails = ['treasury@celsius.network',
'weekly.portfolio@celsius.network']

SCOPES = ['https://www.googleapis.com/auth/drive.readonly']
creds = None
creds = Credentials.from_authorized_user_file('token.json', SCOPES)
service = build('sheets', 'v4', credentials=creds)

# Call the Drive v3 API
SAMPLE_RANGE_NAME = 'Carry Model!A2:H200'
request = service.spreadsheets().values().batchGet(
    spreadsheetId=spreadsheet_id,
    ranges=SAMPLE_RANGE_NAME,
    valueRenderOption='FORMATTED_VALUE')
response = request.execute()
df = pd.DataFrame(response['valueRanges'][0]['values']).reset_index(drop=True)

[['Coin',
  'Base COF',
  '10%Target',
  'Current Utilization',
  'Base',
  'Target Utilization',
  'Carry',
  'Carry Tier 5'],
 ['1INCH',
  '4.81%',
  '0.05292639216',
  '98.77%',
  '2.41%',
  '100.00%',
  '5.22%',
  '6.42%'],
 ['AAVE',
  '4.13%',
  '0.04547844152',
  '93.02%',
  '2.07%',
  '100.00%',
  '4.21%',
  '5.25%'],
 ['AAVE Stable', '3.61%', '1.75', '9.15%', '', '???', None, None],
 ['ADA',
  '2.33%',
  '0.02558055461',
  '93.18%',
  '1.16%',
  '100.00%',
  '2.37%',
  '2.96%'],
 ['ALCX', '2.00%', '0.022', '100.00%', '1.00%', '100.00%', '2.20%', '2.70%'],
 ['ALPHA', '2.00%', '0.022', '100.00%', '1.00%', '100.00%', '2.20%', '2.70%'],
 ['AVAX',
  '5.59%',
  '0.0614819575',
  '91.41%',
  '2.79%',
  '100.00%',
  '5.60%',
  '6.99%'],
 ['BADGER',
  '15.00%',
  '0.165',
  '99.42%',
  '7.50%',
  '100.00%',
  '16.40%',
  '20.15%'],
 ['BAT',
  '0.98%',
  '0.01073239454',
  '90.02%',
  '0.49%',
  '100.00%',
  '0.96%',
  '1.21%'],
 ['BCH',
  '4.53%',
  '0.0498315221',
  '51.49%',
  '2.27%'

,Coin,Base COF,10%Target,Current Utilization,Base,Target Utilization,Carry,Carry Tier 5
8,1INCH,4.81%,0.05292639216,98.77%,2.41%,100.00%,5.22%,6.42%
9,AAVE,4.13%,0.04547844152,93.02%,2.07%,100.00%,4.21%,5.25%
70,AAVE Stable,3.61%,1.75,9.15%,,???,None,None
16,ADA,2.33%,0.02558055461,93.18%,1.16%,100.00%,2.37%,2.96%
66,ALCX,2.00%,0.022,100.00%,1.00%,100.00%,2.20%,2.70%
...,...,...,...,...,...,...,...,...
21,XRP,1.83%,0.02016549975,81.67%,0.92%,100.00%,1.65%,2.11%
14,XTZ,3.80%,0.04183402953,93.82%,1.90%,100.00%,3.91%,4.86%
24,ZEC,1.47%,0.01620118429,17.52%,0.74%,100.00%,0.76%,1.13%
25,ZRX,1.70%,0.01869399917,88.99%,0.85%,100.00%,1.66%,2.08%


In [16]:
df2 = df.copy()
df2.columns = df2.loc[0]
df2 = df2.loc[1:]
na_count = df2['Carry'].dropna().map(lambda x: ('N/A' in x) | ('#DIV/0!' in x)).sum()
na_count += df2['Carry Tier 5'].dropna().map(lambda x: ('N/A' in x) | ('#DIV/0!' in x)).sum()
if na_count>0:
    print("Send_MAIL")
else:
    print("PUSH to DEPLOYABLE")
    df.columns = df.loc[0]
    df = df.loc[1:].reset_index(drop=True)
    idx_drop = df[df['Coin'].isin(['', None, '#NA!','#Value!'])].index.tolist()
    df = df.drop(idx_drop, axis=0).sort_values("Coin")
    data = [df.columns.tolist()] + df.fillna('na').values.tolist()

Send_MAIL
